In [ ]:
# code
import ipywidgets as widgets
from trackanalysis                      import *
from eventdetection.processor.alignment import MeasureEndAlignmentTask
from peakfinding.processor.projection   import PeakProjectorTask
from peakcalling.tohairpin import PiecesPeakGridFit, Pivot, matchpeaks

Tasks.cleaning.let(minextent = 0.05, maxsaturation = 100., maxpingpong = 10)

def beads(track, *tasks, distance = MeasureEndAlignmentTask.distance):
    "returns beads with some tasks applied"
    if track.tasks.subtraction is None:
        track.tasks.subtraction = track.cleaning.fixed()
    return track.apply(
        track.tasks.subtraction,
        Tasks.cleaning,
        MeasureEndAlignmentTask(distance = distance),
        *tasks,
        copy = True
    )

def peaks(track, **kwa):
    "returns peaks with some tasks applied"
    return beads(track, PeakProjectorTask(**kwa))

class TrackDisplayer:
    "display tracks"
    def __init__(self, tracks):
        self.tracks = tracks
        self.out    = widgets.Output(layout={'border': '1px solid black'})
        self.trackw = widgets.Dropdown(
            options = [""],
            value   = "",
            description = "Track:",
            disabled    = False
        )
        self.trackw.on_displayed(self._ondisplayedtrack)
        self.trackw.observe(self._onchangetrack, names = "value")
        self.beadw = widgets.Dropdown(
            options= ["2"],
            value='2',
            description='Bead:',
            disabled= False,
        )
        self.beadw.observe(self._onchangebead, names = "value")
        self.good  = {}
        self.keys  = '', 0
        self.width = 500, 300

    def display(self):
        "display the graphs & the widgets"
        return widgets.VBox([widgets.HBox([self.trackw, self.beadw]), self.out])

    def draw(self, key, bead):
        "display peaks & cycles"
        track = self.tracks[key]
        dispbeads = lambda x: (
            beads(track, **({} if x else {'distance': None}))
            [...,...]
            .withphases(1,8)
            .display.display()[bead]
            .Curve.I.options(line_color = 'deepskyblue' if x else 'red')
        )
        return (
            (dispbeads(False) * dispbeads(True)).options(width = self.width[0])
            + peaks(track).display(zero = False).display()[bead].options(invert_axes = True, width= self.width[1])
        )

    def _ondisplayedtrack(self, _):
        self.trackw.options = list(self.tracks)
        self.trackw.value   = self.trackw.options[0]
        self._onchangetrack({"new": self.trackw.options[0]})

    def _onchangetrack(self, change):
        key = change['new']
        if key not in self.good:
            track = self.tracks[key]
            if track.tasks.subtraction is None:
                track.tasks.subtraction = track.cleaning.fixed()
            self.good[key]  = [str(i) for i in track.cleaning.good()]
        self.beadw.options  = self.good[key]
        self.beadw.value    = self.beadw.options[0]
        self._onchangebead(_)
        
    def _onchangebead(self, _):
        keys = self.trackw.value, int(self.beadw.value)
        if keys != self.keys:
            self.keys = keys
            self.out.clear_output()
            with self.out:
                if len(self.beadw.options) == 0:
                    display(hv.Text(0.5, 0.5, "No good beads in this track"))
                else:
                    display(self.draw(*keys))

class FittedTrackDisplayer(TrackDisplayer):
    "display fitted tracks"
    def __init__(self, tracks, **kwa):
        super().__init__(tracks)
        self.delta  = sum(abs(i) for i in kwa.pop("range", (0, 0)))
        self.fitter = self.instantiatefitter(**kwa)

    def draw(self, key, bead):
        "display peaks & data"
        pks = peaks(self.tracks[key])
        dtl = pks.detailed(bead)
        lst = pks[bead]['peaks']
        opt = self.fitter.optimize(lst)
        dtl.details.transform(opt[0][1:])
        out = hv.Overlay([
            dtl.display(zero = False).display(),
            *(
                hv.Scatter((i, 0.1)).options(size = 10)
                for i in self.fitter.reconstruct(opt)
            )
        ])

        theo  = np.concatenate([i.peaks for i in self.fitter.pieces])
        theo -= self.fitter.pieces[0].peaks[-1]
        rec   = np.concatenate(self.fitter.reconstruct(opt))
        rec  -= self.fitter.pieces[0].peaks[-1]
        ids   = [
            matchpeaks(i.peaks, (lst-j[2])*j[1], self.fitter.window)
            for i, j in zip(self.fitter.pieces, opt)
        ]
        exp   = [
            np.full(len(i.peaks), np.NaN, dtype = 'f4')
            for i in self.fitter.pieces
        ]
        rsize = np.diff(rec[len(exp[0])-1:][:-1])[0]-self.delta
        for j, k in zip(exp, ids):
            j[k[k>=0]] = lst[k>=0]
        frame = pd.DataFrame({
            'Experiment (µm)': np.concatenate(exp),
            'Theory (base)': theo,
            'Reconstructed (base)': rec
        })
        out   = (
            out.options(
                width = self.width[0],
                title_format = "Repeat region size: {:.0f} bases".format(rsize)
            )
            
            + hv.Table(frame.fillna("")).options(width = self.width[1])
        )
        return out

    @staticmethod
    def instantiatefitter(dna, oligos, div):
        "create the fitter instance"
        seq = next(sequences.read(dna))[1]
        ols = [i.strip() for i in open(oligos).readlines()[:-1]]
        pos = sequences.peaks(seq, ols)
        pos = np.array(list(pos[~pos['orientation']]['position'])+[len(seq)])

        fit = PiecesPeakGridFit(ordered = True)
        fit.appendpiece(
                peaks      = pos[:div],
                stretch    = (1e3, 200, 100),
                strandsize = pos[-1],
        )
        fit.appendpiece(
                peaks      = pos[div:],
                pivot      = Pivot.top,
                stretch    = (1e3, 200, 100),
                strandsize = pos[-1],
        )
        return fit

In [ ]:
DNA    = "/home/pol/Documents/tracks/repeats/fmr1.fa"       # sequence in fasta format
OLIGOS = "/home/pol/Documents/tracks/repeats/OligoList.txt" # list of oligos, 1 per line
DIV    = 7       # the number of peaks below the repeat region
RANGE  = 10, -10 # distance to the repeat region from the last bottom peak and the first top peak

# The tracks:
# * 1st argument is the path to the tracks. Don't forget the '*.trk' at the end
# * 2nd argument is optional. It uses regular expressions to extract a simpler name from track file names.
TRACKS = TracksDict("/data/tracks/repeats/*.trk", match = ".*(FOV\d+).*")
TRACKS['plasmid'] = Track(path = "/data/tracks/repeats/plasmid_FmrI_2018-09-10.trk")

In [ ]:
# Display RAW data
TrackDisplayer(TRACKS).display()

In [ ]:
%%opts Curve[tools=['hover']]
FittedTrackDisplayer(TRACKS, range = RANGE, dna = DNA, oligos = OLIGOS, div = DIV).display()